In [1]:
#This notebook creates samples files for the analysis and can be used to extract population metadata. 
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import json
import pickle

In [28]:
major_file_ids = []
with open('./major_file_ids.txt') as major_id_file:
    for line in major_id_file:
        major_file_ids.append(line.strip())
related_file_ids = []
with open('./1000G_698_related_high_coverage.sequence.txt','r') as related_file:
    for line in related_file:
        if line.startswith('#'):
            continue
        related_file_ids.append(line.strip().split()[-7])
phase1_ids = []
with open('./phase1_ids.txt') as phase1_file:
    for line in phase1_file:
        phase1_ids.append(line.strip())
phase1_p1_ids = []
with open('./plus_v1.txt') as phase1_p1_file:
    for line in phase1_p1_file:
        phase1_p1_ids.append(line.strip())
hgsvc_ids = []
with open('./HGSVC/samples.txt') as hgsvc_file:
    for line in hgsvc_file:
        hgsvc_ids.append(line.strip())
parents_of_phase1_ids = []
with open('./1kGP.3202_samples.pedigree_info.txt') as  pedigree_file:
    pedigree_file.readline()
    for line in pedigree_file:
        data = line.strip().split()
        if data[0] in phase1_ids:
            parents_of_phase1_ids.append(data[1])
            parents_of_phase1_ids.append(data[2])


In [29]:
main_ids = set(major_file_ids)
related_ids = set(related_file_ids)
phase1_ids = set(phase1_ids)
phase1_parents = set(parents_of_phase1_ids)
phase1_p1_ids = set(phase1_p1_ids)
hgsvc_ids = set(hgsvc_ids)


In [30]:
len(hgsvc_ids&phase1_ids),len(hgsvc_ids&phase1_p1_ids),len(phase1_p1_ids&phase1_ids)

(0, 4, 0)

In [31]:
hgsvc_ids&phase1_p1_ids

{'HG00733', 'HG02818', 'HG03486', 'NA19240'}

In [33]:
len(hgsvc_ids), len(hgsvc_ids&main_ids),len(hgsvc_ids&related_ids),len(main_ids),len(hgsvc_ids&main_ids)+len(hgsvc_ids&related_ids)

(69, 32, 32, 2504, 64)

In [36]:
len(phase1_p1_ids), len(phase1_p1_ids&main_ids),len(phase1_p1_ids&related_ids),len(main_ids),len(phase1_p1_ids&main_ids)+len(phase1_p1_ids&related_ids)

(20, 2, 14, 2504, 16)

In [47]:
phase1_p1_agg_ids=phase1_p1_ids|phase1_ids
p1p1a_parents = []
with open('./1kGP.3202_samples.pedigree_info.txt') as  pedigree_file:
    pedigree_file.readline()
    for line in pedigree_file:
        data = line.strip().split()
        if data[0] in phase1_p1_agg_ids:
            p1p1a_parents.append(data[1])
            p1p1a_parents.append(data[2])
p1p1a_parents = set(p1p1a_parents)


In [48]:
len(main_ids - p1p1a_parents-phase1_p1_agg_ids)

2195

In [2]:
subpopdf = pd.read_csv('./subpops.csv',delimiter=',')
subpopdf.dropna(inplace=True)
subpopdf.set_index('sampleid',inplace=True)

In [9]:
phase1 = pd.read_csv('./phase1_ids.txt',header=None)
phase1_plus1 = pd.read_csv('./plus_v1.txt',header=None)

hgsvc = pd.read_csv('./HGSVC/samples.txt',header=None)

In [10]:
phase1.set_index(0,inplace=True)
phase1_plus1.set_index(0,inplace=True)
hgsvc.set_index(0,inplace=True)

In [11]:
size = phase1.shape[0]
phase1 = phase1.join(subpopdf,how='inner')
size2 = phase1.shape[0]
print(f'Phase 1 Size diff: {size-size2}')
size = phase1_plus1.shape[0]
phase1_plus1 = phase1_plus1.join(subpopdf,how='inner')
size2 = phase1_plus1.shape[0]
print(f'Phase 1 +1 Size diff: {size-size2}')
size = hgsvc.shape[0]
hgsvc = hgsvc.join(subpopdf,how='inner')
size2 = hgsvc.shape[0]
print(f'HSVC Size diff: {size-size2}')


Phase 1 Size diff: 0
Phase 1 +1 Size diff: 2
HSVC Size diff: 2


In [35]:
len(main_ids)

2504

In [50]:
with open('oos_ids_2.txt','wt') as oos_id_file:
    for item in (main_ids - p1p1a_parents)-phase1_p1_agg_ids:
        print(f'0 {item}',file=oos_id_file)
with open('./phase1_ids_2.txt','wt') as sample_id_file:
    for item in phase1_p1_agg_ids:
        print(f'0 {item}',file=sample_id_file)

In [52]:
phase1.groupby('popcode').count()

,suppopcode
popcode,
ACB,9
ASW,5
BEB,10
CDX,6
CHB,5
CHS,11
CLM,11
ESN,7
FIN,2


In [54]:
phase1_plus1.groupby('popcode').count()

,suppopcode
popcode,
ACB,2
ASW,2
CLM,1
ESN,1
GWD,2
KHV,1
LWK,1
MSL,2
PJL,1


## Random EUR Samples

In [103]:
eur12_ids = []
with open('./eur/sids_12.txt','r') as proposed_eur_file:
    for line in proposed_eur_file:
        eur12_ids.append(line.strip().split()[1])
eur24_ids = []
with open('./eur/sids_24.txt','r') as proposed_eur_file:
    for line in proposed_eur_file:
        eur24_ids.append(line.strip().split()[1])
all_eur_ids = []
with open('./proposed_eur_ids.txt','r') as proposed_eur_file:
    for line in proposed_eur_file:
        all_eur_ids.append(line.strip())
eur12_ids = set(eur12_ids)
eur24_ids = set(eur24_ids)
all_eur_ids = set(all_eur_ids)

In [101]:
len(eur12_ids-phase1_p1_agg_ids),len(eur24_ids-phase1_p1_agg_ids),len(phase1_p1_ids- phase1_ids),len((eur12_ids-phase1_ids)&phase1_p1_ids),len((eur24_ids-phase1_ids)&phase1_p1_ids),len(all_eur_ids-phase1_p1_agg_ids)


(12, 24, 20, 0, 0, 32)

In [104]:
eur12_ids = eur12_ids-phase1_p1_agg_ids
eur24_ids = eur24_ids-phase1_p1_agg_ids
all_eur_ids = all_eur_ids-phase1_p1_agg_ids
with open('./eur/euro_ids_12.txt','wt') as proposed_eur_file:
    for item in eur12_ids:
        print(item,file=proposed_eur_file)

with open('./eur/euro_ids_24.txt','wt') as proposed_eur_file:
    for item in eur24_ids:
        print(item,file=proposed_eur_file)

with open('./eur/euro_ids_32.txt','wt') as proposed_eur_file:
    for item in all_eur_ids:
        print(item,file=proposed_eur_file)

# with open('./eur/sids.txt','wt') as output_file:
#     for item in all_eur_ids|phase1_p1_agg_ids:
#         print(f'{item} {item}',file=output_file)
# with open('./eur/sids_12.txt','wt') as output_file:
#     for item in eur12_ids|phase1_p1_agg_ids:
#         print(f'{item} {item}',file=output_file)
# with open('./eur/sids_24.txt','wt') as output_file:
#     for item in eur12_ids|phase1_p1_agg_ids:
#         print(f'{item} {item}',file=output_file)

In [106]:
subpopdf.loc[eur12_ids].groupby('popcode').count()

/var/folders/_c/kxtvc9ss3slcnlpn_1b6jtlw0000gp/T/ipykernel_5615/3379991767.py:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  subpopdf.loc[eur12_ids].groupby('popcode').count()


,suppopcode
popcode,
FIN,6
GBR,2
IBS,1
TSI,3


In [107]:
subpopdf.loc[eur24_ids].groupby('popcode').count()

/var/folders/_c/kxtvc9ss3slcnlpn_1b6jtlw0000gp/T/ipykernel_5615/2198654660.py:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  subpopdf.loc[eur24_ids].groupby('popcode').count()


,suppopcode
popcode,
FIN,6
GBR,8
IBS,3
TSI,7


In [108]:
subpopdf.loc[all_eur_ids].groupby('popcode').count()

/var/folders/_c/kxtvc9ss3slcnlpn_1b6jtlw0000gp/T/ipykernel_5615/974550828.py:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  subpopdf.loc[all_eur_ids].groupby('popcode').count()


,suppopcode
popcode,
FIN,8
GBR,8
IBS,8
TSI,8


## AFR Samples generated in evaluate notebook

In [82]:
afk_samples = []
with open('./afk/all_bg.txt','r') as afk_sample_file:
    for line in afk_sample_file:
        afk_samples.append(line.strip())
afk_ids = set(afk_samples)
afk_12_ids = set(afk_samples[-12:])
afk_24_ids = set(afk_samples[-24:])

In [110]:
subpopdf.loc[afk_12_ids].groupby('popcode').count()

/var/folders/_c/kxtvc9ss3slcnlpn_1b6jtlw0000gp/T/ipykernel_5615/838707325.py:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  subpopdf.loc[afk_12_ids].groupby('popcode').count()


,suppopcode
popcode,
ACB,2
ASW,6
ESN,1
MSL,1
YRI,2


In [111]:
subpopdf.loc[afk_24_ids].groupby('popcode').count()

/var/folders/_c/kxtvc9ss3slcnlpn_1b6jtlw0000gp/T/ipykernel_5615/3246670217.py:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  subpopdf.loc[afk_24_ids].groupby('popcode').count()


,suppopcode
popcode,
ACB,4
ASW,14
ESN,1
LWK,2
MSL,1
YRI,2


In [112]:
subpopdf.loc[afk_ids].groupby('popcode').count()

/var/folders/_c/kxtvc9ss3slcnlpn_1b6jtlw0000gp/T/ipykernel_5615/108303483.py:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  subpopdf.loc[afk_ids].groupby('popcode').count()


,suppopcode
popcode,
ACB,6
ASW,15
ESN,4
LWK,2
MSL,1
YRI,4


## Samples chosen using greedy iterative model with no labels

In [83]:
gi_samples = []
with open('./gv2_all_extra.csv','r') as gi_file:
    for line in gi_file:
        gi_samples.append(line.strip())
gi_samples = set(gi_samples)

In [94]:
all_samples = afk_ids | gi_samples | all_eur_ids | phase1_p1_agg_ids

## Writing Samples to files and Writing OOS too
The intersection between all these samples are zero. I checked

In [97]:
with open('./eval/sample_ids.txt','wt') as sample_file:
    for id in all_samples:
        print(f'0 {id}',file=sample_file)
with open('./eval/oos_ids.txt','wt') as oos_file:
    for id in (main_ids - p1p1a_parents)-all_samples:
        print(f'0 {id}',file=oos_file)

[]